In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn import linear_model
from sklearn.model_selection import train_test_split


In [ ]:
df = pd.read_csv("../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df['DEATH_EVENT'].value_counts()

In [ ]:
sns.countplot(df['DEATH_EVENT'])
plt.title('Death Counts')

In [ ]:
df['anaemia'].value_counts()

In [ ]:
sns.countplot(df['anaemia'])
plt.title("Anaemia Count")

In [ ]:
df['diabetes'].value_counts()

In [ ]:
sns.countplot(df['diabetes'])
plt.title('Diabetes Count')

In [ ]:
df['high_blood_pressure'].value_counts()

In [ ]:
sns.countplot(df['high_blood_pressure'])
plt.title('High Blood Pressure Count')

In [ ]:
fig, ax =plt.subplots(ncols=5, figsize = (15,3))
sns.countplot(df['anaemia'], ax=ax[0])
sns.countplot(df['diabetes'], ax=ax[1])
sns.countplot(df['high_blood_pressure'], ax=ax[2])
sns.countplot(df['sex'], ax=ax[3])
sns.countplot(df['smoking'], ax=ax[4])
fig.show()

In [ ]:
df['age'].hist(bins = 15, color = 'c')
plt.title("Age Histogram")

In [ ]:
df['creatinine_phosphokinase'].hist(bins = 30, color = 'c')
plt.title("Creatinine Phosphokinase Histogram")

In [ ]:
df['ejection_fraction'].hist(bins = 13, color = 'c')
plt.title("Ejection Fraction Histogram")

In [ ]:
df['platelets'].hist(bins = 10, color = 'c')
plt.title("Platelets Histogram")

In [ ]:
df['serum_creatinine'].hist(bins = 30, color = 'c')
plt.title("Serum Creatinine Histogram")

In [ ]:
df['serum_sodium'].hist(bins = 30, color = 'c')
plt.title("Serum Sodium Histogram")

In [ ]:
df['time'].hist(bins = 30, color = 'c')
plt.title("Time Histogram")

In [ ]:
corr = df.corr()
corr

In [ ]:
fig, ax = plt.subplots(figsize=(9,9))
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns,
        vmin = -1.0, vmax = 1.0,
        center = 0, annot = False)

# Random Forest

In [ ]:
X = df.drop(["DEATH_EVENT"], axis = 1)

In [ ]:
y = df['DEATH_EVENT']

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
rf = RandomForestClassifier()

In [ ]:
scaler = MinMaxScaler()

In [ ]:
rf_pipe = Pipeline([('Scaler', scaler), ('RandomForest', rf)])

In [ ]:
rf_params = {'RandomForest__n_estimators': [100, 200, 600],
             'RandomForest__max_features': [3,5,6]
             }

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 9)

In [ ]:
rf_grid = GridSearchCV(rf_pipe, param_grid = rf_params, cv = 8, scoring = "accuracy")
rf_grid.fit(X_train, y_train)

In [ ]:
rf_grid.cv_results_
rf_grid_results = pd.DataFrame(rf_grid.cv_results_)

In [ ]:
rf_grid_results

In [ ]:
rf_grid.best_score_

In [ ]:
rf_grid.best_params_

In [ ]:
rf_pred = rf_grid.predict(X_test)

In [ ]:
rf_acc = accuracy_score(y_test, rf_pred)
rf_acc

In [ ]:
rf_conf = confusion_matrix(y_test, rf_pred)
rf_conf

In [ ]:
rf_conf = pd.DataFrame(data = rf_conf, columns = [['Pred:No Failure', 'Pred: Failure']], index = [['Act: No Failure', 'Act: Failure']])

In [ ]:
sns.heatmap(rf_conf, annot = True)

In [ ]:
rf_fn= rf_conf.iloc[1,0]
rf_fn

In [ ]:
rf_tp = rf_conf.iloc[1,1]
rf_tp

In [ ]:
rf_prec = rf_tp/(rf_fn + rf_tp)
rf_prec

# Support Vector Machine

In [ ]:
svc = svm.SVC()

In [ ]:
svc_pipe = Pipeline([('Scaler', scaler), ('svc', svc )])

In [ ]:
gamma_range = range(1,10)

In [ ]:
c_range = range(1,10)

In [ ]:
svc_params = dict(svc__gamma = gamma_range, svc__C = c_range)

In [ ]:
svc_grid = GridSearchCV(svc_pipe, param_grid = svc_params, cv = 8, scoring = "accuracy", n_jobs=-1)

In [ ]:
svc_grid.fit(X_train, y_train)

In [ ]:
svc_results = pd.DataFrame(svc_grid.cv_results_)

In [ ]:
svc_results

In [ ]:
svc_grid.best_score_

In [ ]:
svc_grid.best_params_

In [ ]:
svc_pred = svc_grid.predict(X_test)

In [ ]:
svc_acc = accuracy_score(y_test, svc_pred)
svc_acc

In [ ]:
svc_conf = confusion_matrix(y_test, svc_pred)
svc_conf

In [ ]:
svc_conf = pd.DataFrame(data = svc_conf, columns = [['Pred:No Failure', 'Pred: Failure']], index = [['Act: No Failure', 'Act: Failure']])

In [ ]:
sns.heatmap(svc_conf, annot = True)

In [ ]:
svc_fn = svc_conf.iloc[1,0]
svc_fn

In [ ]:
svc_tp = svc_conf.iloc[1,1]
svc_tp

In [ ]:
svc_prec = svc_tp/(svc_tp + svc_fn)
svc_prec

# Artificial Neural Network

In [ ]:
mlp = MLPClassifier()

In [ ]:
mlp_pipe = Pipeline([('scaler', scaler), ('mlp', mlp)])

In [ ]:
mlp_params = {'mlp__solver': ['sgd'],
              'mlp__hidden_layer_sizes': [(10,10,10),(15,15,15),(20,20,20),(30,30,30),(40,40,40)],
              'mlp__max_iter': [2000,6000,8000,10000]
             }

In [ ]:
mlp_grid = GridSearchCV(mlp_pipe, param_grid = mlp_params, cv=8, scoring = "accuracy", n_jobs=-1)

In [ ]:
mlp_grid.fit(X_train, y_train)

In [ ]:
mlp_results = pd.DataFrame(mlp_grid.cv_results_)

In [ ]:
mlp_results

In [ ]:
mlp_grid.best_score_

In [ ]:
mlp_grid.best_params_

In [ ]:
mlp_pred = mlp_grid.predict(X_test)

In [ ]:
mlp_acc = accuracy_score(y_test, mlp_pred)
mlp_acc

In [ ]:
mlp_conf = confusion_matrix(y_test, mlp_pred)
mlp_conf

In [ ]:
mlp_conf = pd.DataFrame(data = mlp_conf, columns = [['Pred:No Failure', 'Pred: Failure']], index = [['Act: No Failure', 'Act: Failure']])

In [ ]:
sns.heatmap(mlp_conf, annot = True)

In [ ]:
mlp_fn = mlp_conf.iloc[1,0]
mlp_fn

In [ ]:
mlp_tp = mlp_conf.iloc[1,1]
mlp_tp

In [ ]:
mlp_prec = mlp_tp/(mlp_tp + mlp_fn)
mlp_prec

In [ ]:
results = pd.DataFrame({'Model': ['Random Forest',
                                  'Support Vector Machine',
                                  'Artifical Neural Network'],
                        'Accuracy': [rf_acc,
                                     svc_acc,
                                     mlp_acc],
                        'Precision': [rf_prec,
                                    svc_prec,
                                    mlp_prec]
                       })

In [ ]:
results = results.set_index("Model")

In [ ]:
results = results.sort_values(by="Precision", ascending = False)

In [ ]:
ax = results.plot(kind = 'bar', title = "Model Results", figsize = (7,5), legend = True, fontsize = 12)